# Pre-Work

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPool2D
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy
import os
import pandas as pd

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 1GB * 2 of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024 * 2)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


# Selectively Loading Data with Filter

In [3]:
os.chdir('../data/raw')

In [4]:
df = pd.read_csv('driver_imgs_list.csv')

In [5]:
df.head()

,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg
2,p002,c0,img_25094.jpg
3,p002,c0,img_69092.jpg
4,p002,c0,img_92629.jpg


Process the dataframe to make a column that generates the filepathway from this parent folder ot each individual file. Then create a function that will generate the crossvalidation train/test splits, with each iteration pulling out one specific subject.

# Datagen

In [20]:
path = '../data/raw/imgs/train'

In [31]:
scores = numpy.empty([5,2])
checkpoint = ModelCheckpoint('weights.hdf5', \
                             mode='max', \
                             monitor='val_accuracy', \
                             save_best_only=True)
callbacks_list=[checkpoint]    

In [32]:
for i in range(5):
    model = Sequential()
    model.add(Conv2D(10, kernel_size=3, activation='relu', input_shape=(480, 640, 3)))
    model.add(MaxPool2D(30))
    model.add(Flatten())
    model.add(Dense(10, activation='softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    datagen = keras.preprocessing.image.ImageDataGenerator(validation_split=0.3)
    
    train = datagen.flow_from_directory(path, \
                                   target_size=(480, 640), \
                                   subset='training')
    
    val = datagen.flow_from_directory(path, \
                                   target_size=(480, 640), \
                                   subset='validation')
    
    model.fit(train, \
              epochs=3, \
              steps_per_epoch=20, \
              validation_data=val, \
              validation_steps=20, \
              callbacks=callbacks_list)
    
    #print('Evaluating')
    #history = model.evaluate(val)
    #scores[i,0] = history[0]
    #scores[i,1] = history[1]
    

Found 15702 images belonging to 10 classes.
Found 6722 images belonging to 10 classes.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 20 steps, validate for 20 steps
Epoch 1/3
20/20 [==============================] - 29s 1s/step - loss: 184.2983 - accuracy: 0.1031 - val_loss: 108.1223 - val_accuracy: 0.1328
Epoch 2/3
20/20 [==============================] - 32s 2s/step - loss: 55.8650 - accuracy: 0.1312 - val_loss: 27.2335 - val_accuracy: 0.1984
Epoch 3/3
20/20 [==============================] - 22s 1s/step - loss: 18.8952 - accuracy: 0.2156 - val_loss: 13.3917 - val_accuracy: 0.2609
Found 15702 images belonging to 10 classes.
Found 6722 images belonging to 10 classes.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 20 steps, validate for 20 steps
Epoch 1/3
20/20 [==============================] - 25s 1s/step - loss: 145.4389 - accuracy: 0.1016 - val_loss: 73.0403 - val_accuracy: 0.1375
Epoch 2/3
20/20 [==============================] - 21s 1s/step - loss: 41.4